# Spark 3.0 Tutorial
- Author: Akira Takihara Wang (https://github.com/akiratwang)
- Tutorial Up-to-Date as of: April 2021  
- Usage: For MAST30034 students only  

Tutorial Operating System(s):
- Windows 10 and WSL2
- Linux

# Working with Larger Datasets with a Scalable Solution!
- Consider the full 2015 Taxi Dataset (at ~2GB per month @ ~24GB annually)
- Datasets with more than 20k rows would be hard for Excel, but fine for Pandas.
- A file with more than 100mil rows (a few GB) is large for Pandas.
- Although `pandas` would be sufficient for each month, how about a whole year?

That's right, use Spark 3.0!
![image.png](https://spark.apache.org/images/spark-logo-trademark.png)


## Pre-Requisites for this Tutorial
1. You must already have Spark installed.
2. You need the dataset downloaded.

The code below downloads all 2015 data directly from the Amazon S3 Bucket. This is approximately ~21.3GB in size, so make sure you have ample storage space. You will only need to run this once.
```python
from os.path import getsize
from urllib.request import urlretrieve

output_dir = "../data/large"
fname_template = "yellow_tripdata_2015"

for m in range(1, 13):
    month = str(m).zfill(2)
    out = f'{fname_template}-{month}.csv'
    url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/{out}"
    urlretrieve(url, f"{output_dir}/{out}")

    print(f"Done downloading {out} to {output_dir} with size {getsize(f'{output_dir}/{out}') / 1073741824:.2f}GB")
```

## Optional Installation
- Requires NodeJS and nbextensions installed:
```bash
# install NodeJS
sudo apt install npm
# install Jupyter Extensions
pip3 install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
jupyter nbextension enable varInspector/main
```
- Follow instructions to install `SparkMonitor` (https://github.com/swan-cern/jupyter-extensions)
```bash
pip3 install sparkmonitor
jupyter nbextension install sparkmonitor --py --user
jupyter nbextension enable  sparkmonitor --py --user
jupyter serverextension enable --py --system sparkmonitor  --user
jupyter lab build
ipython profile create
echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.py
```

![image.gif](https://user-images.githubusercontent.com/6822941/29753710-ff8849b6-8b94-11e7-8f9c-bdc59bf72143.gif)

Only run this cell below if you have installed `SparkMonitor`. Otherwise, it will result in an error.

In [1]:
from pyspark import SparkContext

# Start the spark context
sc = SparkContext.getOrCreate(conf=swan_spark_conf) 

# Starting a Spark Session
Start your Spark Session using `SparkSession.builder.getOrCreate()`. This is an object that provides some point of entry to interact with Spark functionalities.

In [2]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

# create a spark session (which will run spark jobs)
spark = SparkSession.builder.getOrCreate()

## Spark DataFrames
To create a Spark DataFrames using a Pandas, simply pass it through `spark.createDataFrame()`.
- It's common convention to name pandas df as `df` and spark df as `sdf`
- And yes, Spark DataFrames *do* look ugly...

In [3]:
import pandas as pd

df = pd.read_csv('../data/sample.csv')

In [4]:
sdf = spark.createDataFrame(df)
sdf.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|RatecodeID|store_and_fwd_flag|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2|        1/12/15 0:00|         1/12/15 0:05|              5|         0.96|    -73.97994232|    40.76538086|         1|                 N|     -73.96630

- If you want to make it look nice (for the first 20 rows), then you can change the setting.
- Use `sdf.limit()` as the alternative to `df.head()` from Pandas

In [5]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [6]:
sdf.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,1/12/15 0:00,1/12/15 0:05,5,0.96,-73.97994232,40.76538086,1,N,-73.96630859,40.76308823,1,5.5,0.5,0.5,1.0,0.0,0.3,7.8
2,1/12/15 0:00,1/12/15 0:00,2,2.69,-73.97233582,40.76237869,1,N,-73.99362946,40.74599838,1,21.5,0.0,0.5,3.34,0.0,0.3,25.64
2,1/12/15 0:00,1/12/15 0:00,1,2.62,-73.96884918,40.76453018,1,N,-73.97454834,40.79164124,1,17.0,0.0,0.5,3.56,0.0,0.3,21.36
1,1/12/15 0:00,1/12/15 0:05,1,1.2,-73.99393463,40.74168396,1,N,-73.99766541,40.74746704,1,6.5,0.5,0.5,0.2,0.0,0.3,8.0
1,1/12/15 0:00,1/12/15 0:09,2,3.0,-73.98892212,40.72698975,1,N,-73.97559357,40.6968689,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3


To convert a Spark DataFrame back into a Pandas dataframe:

In [7]:
sdf.toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,1/12/15 0:00,1/12/15 0:05,5,0.96,-73.979942,40.765381,1,N,-73.966309,40.763088,1,5.5,0.5,0.5,1.00,0.0,0.3,7.80
1,2,1/12/15 0:00,1/12/15 0:00,2,2.69,-73.972336,40.762379,1,N,-73.993629,40.745998,1,21.5,0.0,0.5,3.34,0.0,0.3,25.64
2,2,1/12/15 0:00,1/12/15 0:00,1,2.62,-73.968849,40.764530,1,N,-73.974548,40.791641,1,17.0,0.0,0.5,3.56,0.0,0.3,21.36
3,1,1/12/15 0:00,1/12/15 0:05,1,1.20,-73.993935,40.741684,1,N,-73.997665,40.747467,1,6.5,0.5,0.5,0.20,0.0,0.3,8.00
4,1,1/12/15 0:00,1/12/15 0:09,2,3.00,-73.988922,40.726990,1,N,-73.975594,40.696869,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2,4/12/15 22:55,4/12/15 23:03,1,0.75,-73.994370,40.746239,1,N,-73.980774,40.742146,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80
99996,1,4/12/15 22:55,4/12/15 23:08,1,2.40,-73.968346,40.759735,1,N,-73.969879,40.784767,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30
99997,1,4/12/15 22:55,4/12/15 23:01,1,0.80,-73.993484,40.742168,1,N,-73.984390,40.750008,1,6.0,0.5,0.5,1.45,0.0,0.3,8.75
99998,2,4/12/15 22:55,4/12/15 23:17,1,4.73,-73.984993,40.747929,1,N,-73.981552,40.689789,1,18.5,0.5,0.5,3.96,0.0,0.3,23.76


- Now, you might realize that this is still redundant as you need to read it in using Pandas with this method.
- Likewise, using a `feather` dataset format requires you to read it into Pandas and then into Spark. 

## Overcoming Dataset Formats
- Directly use Apache Arrow (framework that `feather` is built on) with `pip3 install pyarrow`
- Set `spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)`

In [8]:
# Benchmark Normal
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', False)
%time sdf.toPandas()

# Benchmark with Apache Arrow
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
%time sdf.toPandas()

CPU times: user 500 ms, sys: 109 ms, total: 609 ms
Wall time: 932 ms
CPU times: user 62.5 ms, sys: 46.9 ms, total: 109 ms
Wall time: 670 ms


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,1/12/15 0:00,1/12/15 0:05,5,0.96,-73.979942,40.765381,1,N,-73.966309,40.763088,1,5.5,0.5,0.5,1.00,0.0,0.3,7.80
1,2,1/12/15 0:00,1/12/15 0:00,2,2.69,-73.972336,40.762379,1,N,-73.993629,40.745998,1,21.5,0.0,0.5,3.34,0.0,0.3,25.64
2,2,1/12/15 0:00,1/12/15 0:00,1,2.62,-73.968849,40.764530,1,N,-73.974548,40.791641,1,17.0,0.0,0.5,3.56,0.0,0.3,21.36
3,1,1/12/15 0:00,1/12/15 0:05,1,1.20,-73.993935,40.741684,1,N,-73.997665,40.747467,1,6.5,0.5,0.5,0.20,0.0,0.3,8.00
4,1,1/12/15 0:00,1/12/15 0:09,2,3.00,-73.988922,40.726990,1,N,-73.975594,40.696869,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2,4/12/15 22:55,4/12/15 23:03,1,0.75,-73.994370,40.746239,1,N,-73.980774,40.742146,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80
99996,1,4/12/15 22:55,4/12/15 23:08,1,2.40,-73.968346,40.759735,1,N,-73.969879,40.784767,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30
99997,1,4/12/15 22:55,4/12/15 23:01,1,0.80,-73.993484,40.742168,1,N,-73.984390,40.750008,1,6.0,0.5,0.5,1.45,0.0,0.3,8.75
99998,2,4/12/15 22:55,4/12/15 23:17,1,4.73,-73.984993,40.747929,1,N,-73.981552,40.689789,1,18.5,0.5,0.5,3.96,0.0,0.3,23.76


As you can see, Apache Arrow is *magnitudes* faster!

## Reading in directly to Spark
Use `spark.read`, where you can pass through either:
- A single file;
- comma separated file names;
- or a folder directory with files.

Below, we read all csv's in 2015 with a dataset size of 20GB+!

In [9]:
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

sdf = spark.read.csv('../data/large', header=True)

In [10]:
f"{sdf.count():,} rows!"

'2,580,791 rows!'

Damn, ain't it great that you can read in all the csv's without having to append or merge them **AND** no `MemoryError`???

## Schema
- It is best practice to create a standard **schema** for your dataset. 
- It's very similar to creating a table in SQL (in fact its based on this) where you must specify what datatype the column is prior to adding data values.
- View all data types here: https://spark.apache.org/docs/latest/sql-ref-datatypes.html


Note: `RatecodeID` and `RateCodeID` are the same column, but inconsistent across months. We will be renaming it.

In [11]:
import pyspark.sql.functions as F

from pyspark.sql.types import *
from pyspark.sql.functions import col

In [12]:
ints = ('VendorID', 'passenger_count', 'RateCodeID', 'RatecodeID','payment_type')
doubles = ('trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
           'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount')
strings = ('store_and_fwd_flag',)
dtimes = ('tpep_pickup_datetime', 'tpep_dropoff_datetime', )

dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})

In [13]:
schema = StructType()

for column in sdf.columns:
    schema.add(column, # column name
               dtypes[column], # data type
               True # is nullable?
              )

In [14]:
sdf_with_schema = spark.read.csv('../data/large', header=True, schema=schema) \
    .withColumnRenamed("RatecodeID","RateCodeID") # rename the wrong column

sdf_with_schema.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [15]:
sdf.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



Although most PySpark operations will automatically handle incorrect data types, it is not recommended to rely on this from a Data Integrity standpoint. Schema's will be used by Business Analysts when describing or explaining the whole data pipeline. If the schema is incorrect or suspect to change, then many things can fall apart down the end of the pipeline!

In [16]:
sdf_with_schema.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,N,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.3,-74.00164794921875,40.7242431640625,1,N,-73.99441528320312,40.75910949707031,1,14.5,0.5,0.5,2.0,0.0,0.3,17.8
1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.8,-73.96334075927734,40.80278778076172,1,N,-73.95182037353516,40.82441329956055,2,9.5,0.5,0.5,0.0,0.0,0.3,10.8
1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.5,-74.00908660888672,40.71381759643555,1,N,-74.00432586669922,40.71998596191406,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8
1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.0,-73.97117614746094,40.762428283691406,1,N,-74.00418090820312,40.742652893066406,2,15.0,0.5,0.5,0.0,0.0,0.3,16.3


## Transformations and Lazy Evaluation (IMPORTANT)
- Transformations transform a Spark DataFrame into a new DataFrame *without* altering the original data, making Spark **immutable**.
- For example, operations will return transformed results rather than mutating the original. 
- It's common to see `sdf = sdf.some_transformation()` if you are looking to overwrite it.
- Finally, all operations in Spark are evaluated lazily! That is, the data doesn't "move" until called upon.

Take for example the code block below. Even when renaming columns, we need to overwrite the original variable.

In [17]:
sdf_with_schema = sdf_with_schema.withColumnRenamed('tpep_pickup_datetime', 'pickup_time') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_time')

In [18]:
sdf_with_schema.limit(5)

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,N,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.3,-74.00164794921875,40.7242431640625,1,N,-73.99441528320312,40.75910949707031,1,14.5,0.5,0.5,2.0,0.0,0.3,17.8
1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.8,-73.96334075927734,40.80278778076172,1,N,-73.95182037353516,40.82441329956055,2,9.5,0.5,0.5,0.0,0.0,0.3,10.8
1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.5,-74.00908660888672,40.71381759643555,1,N,-74.00432586669922,40.71998596191406,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8
1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.0,-73.97117614746094,40.762428283691406,1,N,-74.00418090820312,40.742652893066406,2,15.0,0.5,0.5,0.0,0.0,0.3,16.3


## DataType Conversions
Consider `tpep_pickup_datetime` as a `StringType()` and wish to convert to a `TimeStampType()`:
```python
# Method 1 using withColumn()
sdf_with_schema.withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime").cast(TimestampType()))

# Method 2 using select()
sdf_with_schema.select(col("tpep_pickup_datetime").cast("Timestamp"))

# Method 3 using selectExpr() - similar to SQL syntax
sdf_with_schema.selectExpr("cast(tpep_pickup_datetime as timestamp)")
```

For an actual example, let's take a look at the `store_and_fwd_flag` which should be boolean. Currently, we have `N` and `Y` which we can resolve by assigning the column to a boolean condition.

In [19]:
sdf_with_schema_temp = sdf_with_schema.withColumn("store_and_fwd_flag_bool", 
                                             (sdf_with_schema["store_and_fwd_flag"] == 'Y') \
                                             .cast("boolean"))

In [20]:
sdf_with_schema_temp.select('store_and_fwd_flag', 'store_and_fwd_flag_bool').limit(5)

store_and_fwd_flag,store_and_fwd_flag_bool
N,false
N,false
N,false
N,false
N,false


As you can see, we now have the string `store_and_fwd_flag` set to boolean now under `store_and_fwd_flag_bool`. Let's change it in the `sdf_with_schema` DataFrame.

In [21]:
sdf_with_schema = sdf_with_schema.withColumn("store_and_fwd_flag", 
                                             (sdf_with_schema["store_and_fwd_flag"] == 'Y') \
                                             .cast("boolean"))

## Retrieving and Filtering Data 
Collecting:
- The `collect()` method is an operation that _collects_ all the rows for you (recall that Spark has lazy evaluation, so this method is the evaluation step).
- If you use `collect()` on the full dataset or large partition, you will still result in an `OutOfMemoryError` as it will need to bring it into memory.
- If you want to just get the size of the result, you can use `count()`.

Filtering:
- Similar to the syntax with `df.loc[]` from Pandas.
- Use bitwise `&` or `|` to filter based on several conditions.
- If you want to use NumPy's `.isin()` method, it's the same for Spark (and bitwise not `~` for the not in)

In [22]:
small_sdf = sdf_with_schema.limit(1000)

In [23]:
rows = small_sdf.select('total_amount').limit(5).collect()
rows

[Row(total_amount=17.05),
 Row(total_amount=17.8),
 Row(total_amount=10.8),
 Row(total_amount=4.8),
 Row(total_amount=16.3)]

In [24]:
# you can index your rows like normal lists
rows[0][0]

17.05

In [25]:
small_sdf.filter(small_sdf.store_and_fwd_flag == True)

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2015-01-20 22:49:11,2015-01-20 23:16:03,1,16.3,-73.78917694091797,40.64181900024414,1,true,-73.96247100830078,40.7198600769043,1,45.0,0.5,0.5,11.57,0.0,0.3,57.87
1,2015-01-26 13:17:59,2015-01-26 13:34:33,1,3.4,-73.9676742553711,40.80243682861328,1,true,-73.96623229980469,40.76617431640625,2,14.0,0.0,0.5,0.0,0.0,0.3,14.8
1,2015-01-01 01:08:56,2015-01-01 01:08:56,4,0.0,-73.98078918457031,40.676605224609375,1,true,0.0,0.0,2,20.0,0.5,0.5,0.0,0.0,0.0,21.3
1,2015-01-07 19:57:52,2015-01-07 20:26:08,1,6.9,-73.98433685302734,40.7369270324707,1,true,-74.0096664428711,40.6494255065918,1,25.0,1.0,0.5,6.7,0.0,0.3,33.5
1,2015-01-23 16:51:38,2015-01-23 16:57:37,3,0.4,-73.96159362792969,40.765174865722656,1,true,-73.95663452148438,40.766239166259766,1,5.5,1.0,0.5,1.45,0.0,0.3,8.75
1,2015-01-10 19:12:24,2015-01-10 19:17:30,1,1.0,-73.97574615478516,40.75114059448242,1,true,-73.96610260009766,40.762516021728516,1,6.0,0.0,0.5,0.82,0.0,0.3,7.62
1,2015-01-10 19:37:09,2015-01-10 19:43:46,1,0.9,-73.98411560058594,40.72096252441406,1,true,-73.9983901977539,40.72467041015625,2,6.5,0.0,0.5,0.0,0.0,0.3,7.3
1,2015-01-03 08:58:44,2015-01-03 09:19:31,2,10.8,-73.98988342285156,40.746803283691406,1,true,-73.86493682861328,40.770469665527344,2,31.0,0.0,0.5,0.0,5.33,0.0,37.13
1,2015-01-07 20:40:03,2015-01-07 20:45:48,1,0.8,-73.9742431640625,40.76160430908203,1,true,-73.98139190673828,40.753517150878906,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8


You can also filter DataFrame rows using `startswith()`, `endswith()`, and `contains()` 

In [26]:
# all trips whose pickups are -74.X, 40.Y
small_sdf.filter((small_sdf.pickup_longitude.startswith('-74.')) 
                 & (small_sdf.pickup_latitude.startswith('40.')))

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.3,-74.00164794921875,40.7242431640625,1,false,-73.99441528320312,40.75910949707031,1,14.5,0.5,0.5,2.0,0.0,0.3,17.8
1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.5,-74.00908660888672,40.71381759643555,1,false,-74.00432586669922,40.71998596191406,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8
1,2015-01-10 20:33:39,2015-01-10 20:42:20,3,0.8,-74.0026626586914,40.7341423034668,1,false,-73.99501037597656,40.72632598876953,1,7.0,0.5,0.5,1.66,0.0,0.3,9.96
1,2015-01-10 20:33:41,2015-01-10 20:35:23,1,0.3,-74.00836181640625,40.704376220703125,1,false,-74.00977325439453,40.707725524902344,2,3.0,0.5,0.5,0.0,0.0,0.3,4.3
1,2015-01-10 20:33:41,2015-01-10 20:39:23,1,1.1,-74.00672149658203,40.73177719116211,1,false,-73.9952163696289,40.73989486694336,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3
2,2015-01-15 19:05:41,2015-01-15 19:10:55,1,0.96,-74.00093841552734,40.74706268310547,1,false,-74.0035629272461,40.735511779785156,1,5.5,1.0,0.5,1.3,0.0,0.3,8.6
2,2015-01-15 19:05:41,2015-01-15 19:12:36,2,1.25,-74.00277709960938,40.717891693115234,1,false,-74.00791931152344,40.704219818115234,1,6.5,1.0,0.5,1.5,0.0,0.3,9.8
1,2015-01-10 19:12:21,2015-01-10 19:31:33,1,2.9,-74.0030517578125,40.727718353271484,1,false,-73.97603607177734,40.76396942138672,2,13.5,0.0,0.5,0.0,0.0,0.3,14.3
2,2015-01-25 00:13:07,2015-01-25 00:41:19,1,6.17,-74.00163269042969,40.740936279296875,1,false,-73.91980743408203,40.743080139160156,1,24.0,0.5,0.5,2.7,0.0,0.3,28.0
1,2015-01-04 13:44:53,2015-01-04 13:51:53,3,1.5,-74.01420593261719,40.71208953857422,1,false,-74.0010986328125,40.72412872314453,2,7.5,0.0,0.5,0.0,0.0,0.0,8.3


Even better, you can use the SQL `LIKE` syntax!
- `like()` for the SQL `LIKE`;
- and `rlike()` for regex matching.

In [27]:
# using SQL LIKE
small_sdf.filter(small_sdf.pickup_time.like('%19:%%:%%'))

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,false,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
2,2015-01-15 19:05:39,2015-01-15 19:32:00,1,2.38,-73.97642517089844,40.739810943603516,1,false,-73.98397827148438,40.75788879394531,1,16.5,1.0,0.5,4.38,0.0,0.3,22.68
2,2015-01-15 19:05:40,2015-01-15 19:21:00,5,2.83,-73.96870422363281,40.75424575805664,1,false,-73.95512390136719,40.78685760498047,2,12.5,1.0,0.5,0.0,0.0,0.3,14.3
2,2015-01-15 19:05:40,2015-01-15 19:28:18,5,8.33,-73.8630599975586,40.76958084106445,1,false,-73.95271301269531,40.78578186035156,1,26.0,1.0,0.5,8.08,5.33,0.3,41.21
2,2015-01-15 19:05:41,2015-01-15 19:20:36,1,2.37,-73.94554138183594,40.779422760009766,1,false,-73.98085021972656,40.78608322143555,1,11.5,1.0,0.5,0.0,0.0,0.3,13.3
2,2015-01-15 19:05:41,2015-01-15 19:20:22,2,7.13,-73.87445831298828,40.774009704589844,1,false,-73.95237731933594,40.718589782714844,1,21.5,1.0,0.5,4.5,0.0,0.3,27.8
2,2015-01-15 19:05:41,2015-01-15 19:31:00,1,3.6,-73.97660064697266,40.751895904541016,1,false,-73.9989242553711,40.714595794677734,2,17.5,1.0,0.5,0.0,0.0,0.3,19.3
2,2015-01-15 19:05:41,2015-01-15 19:10:22,1,0.89,-73.99495697021484,40.745079040527344,1,false,-73.99993896484375,40.734649658203125,1,5.5,1.0,0.5,1.62,0.0,0.3,8.92
2,2015-01-15 19:05:41,2015-01-15 19:10:55,1,0.96,-74.00093841552734,40.74706268310547,1,false,-74.0035629272461,40.735511779785156,1,5.5,1.0,0.5,1.3,0.0,0.3,8.6
2,2015-01-15 19:05:41,2015-01-15 19:12:36,2,1.25,-74.00277709960938,40.717891693115234,1,false,-74.00791931152344,40.704219818115234,1,6.5,1.0,0.5,1.5,0.0,0.3,9.8


In [28]:
# using regex
small_sdf.filter(small_sdf.pickup_time.rlike(r'.+\s(19):\d{2}:\d{2}'))

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,false,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
2,2015-01-15 19:05:39,2015-01-15 19:32:00,1,2.38,-73.97642517089844,40.739810943603516,1,false,-73.98397827148438,40.75788879394531,1,16.5,1.0,0.5,4.38,0.0,0.3,22.68
2,2015-01-15 19:05:40,2015-01-15 19:21:00,5,2.83,-73.96870422363281,40.75424575805664,1,false,-73.95512390136719,40.78685760498047,2,12.5,1.0,0.5,0.0,0.0,0.3,14.3
2,2015-01-15 19:05:40,2015-01-15 19:28:18,5,8.33,-73.8630599975586,40.76958084106445,1,false,-73.95271301269531,40.78578186035156,1,26.0,1.0,0.5,8.08,5.33,0.3,41.21
2,2015-01-15 19:05:41,2015-01-15 19:20:36,1,2.37,-73.94554138183594,40.779422760009766,1,false,-73.98085021972656,40.78608322143555,1,11.5,1.0,0.5,0.0,0.0,0.3,13.3
2,2015-01-15 19:05:41,2015-01-15 19:20:22,2,7.13,-73.87445831298828,40.774009704589844,1,false,-73.95237731933594,40.718589782714844,1,21.5,1.0,0.5,4.5,0.0,0.3,27.8
2,2015-01-15 19:05:41,2015-01-15 19:31:00,1,3.6,-73.97660064697266,40.751895904541016,1,false,-73.9989242553711,40.714595794677734,2,17.5,1.0,0.5,0.0,0.0,0.3,19.3
2,2015-01-15 19:05:41,2015-01-15 19:10:22,1,0.89,-73.99495697021484,40.745079040527344,1,false,-73.99993896484375,40.734649658203125,1,5.5,1.0,0.5,1.62,0.0,0.3,8.92
2,2015-01-15 19:05:41,2015-01-15 19:10:55,1,0.96,-74.00093841552734,40.74706268310547,1,false,-74.0035629272461,40.735511779785156,1,5.5,1.0,0.5,1.3,0.0,0.3,8.6
2,2015-01-15 19:05:41,2015-01-15 19:12:36,2,1.25,-74.00277709960938,40.717891693115234,1,false,-74.00791931152344,40.704219818115234,1,6.5,1.0,0.5,1.5,0.0,0.3,9.8


## Unique Values, Duplicates, and Missing Values
- You can easily grab unique values using `sdf.distinct()` and drop duplicates using `sdf.dropDuplicates()`.
- For missing values, it's the same with Pandas `.fillna()`.

In [29]:
small_sdf.select('passenger_count').distinct()

passenger_count
1
3
2
5
6
4


Here's a code snippet which inserts a `null` value into `total_amount`, then finds the number `nulls` present in `total_amount`. This is because the dataset has no nulls present.
- We should get 1 instance back as there are no `nulls` except the one we inserted.
- As you can see, granular changes with Spark requires a round-a-bout approach!

In [30]:
# create a single row aand convert to sdf
temp = small_sdf.limit(1).toPandas()
temp.rename({'pickup_time': 'tpep_pickup_datetime', 'dropoff_time': 'tpep_dropoff_datetime'}, axis=1, inplace=True)
temp['total_amount'] = None
r = spark.createDataFrame(temp, schema)
r = r.withColumn("store_and_fwd_flag",(r["store_and_fwd_flag"] == 'true').cast("boolean"))

# take the union of the two (that is, add the row to small_sdf)
small_sdf = small_sdf.union(r)

# sample 5 random rows from 10% of all the data
small_sdf.sample(0.1).limit(5)

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2015-01-10 20:33:39,2015-01-10 20:42:20,3,0.8,-74.0026626586914,40.7341423034668,1,false,-73.99501037597656,40.72632598876953,1,7.0,0.5,0.5,1.66,0.0,0.3,9.96
2,2015-01-15 19:05:41,2015-01-15 19:20:22,2,7.13,-73.87445831298828,40.774009704589844,1,false,-73.95237731933594,40.718589782714844,1,21.5,1.0,0.5,4.5,0.0,0.3,27.8
2,2015-01-15 19:05:43,2015-01-15 19:17:43,2,2.23,-73.98509216308594,40.756988525390625,1,false,-73.98487854003906,40.7801513671875,1,10.0,1.0,0.5,2.2,0.0,0.3,14.0
1,2015-01-10 19:12:21,2015-01-10 19:46:08,1,16.4,-73.97709655761719,40.7631721496582,2,false,-73.79071044921875,40.6442756652832,1,52.0,0.0,0.5,10.0,0.0,0.3,62.8
2,2015-01-25 00:13:06,2015-01-25 00:18:30,1,1.18,-73.98302459716797,40.767818450927734,1,false,-73.96830749511719,40.76314926147461,1,6.0,0.5,0.5,1.8,0.0,0.3,9.1


In [31]:
# now, find all row values that are nan or null for total_amount
small_sdf.where(col("total_amount").isNull())

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,false,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,null


## Sorting Values
- As you should know from 2nd year Algorithms, sorting algorithms whilst efficient, are still costly!
- Even for a fast `O(nlogn)` sorting algorithm, our dataset will require `146,000,000 x log2(146,000,000) = 146,000,000 x 27.12` operations!
- So, be careful when you run a sort - avoid it if you can.

Here's a few ways to sort a column:
```python
# sort by total amount from largest and passenger count from smallest
small_sdf.sort(small_sdf.total_amount.desc(), small_sdf.passenger_count.asc())
small_sdf.sort(col("total_amount").desc(), col("passenger_count").asc())
small_sdf.orderBy(col("total_amount").desc(), col("passenger_count").asc())
```

In [32]:
small_sdf.sort(col("total_amount").desc(), col("passenger_count").asc())

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2015-01-23 09:45:36,2015-01-23 10:20:37,1,18.06,-73.97798156738281,40.76349639892578,3,false,-74.17701721191406,40.694862365722656,1,68.0,0.0,0.0,16.44,13.9,0.3,98.64
1,2015-01-26 12:41:21,2015-01-26 13:03:26,2,16.3,-73.9983901977539,40.761680603027344,3,false,-74.17723846435547,40.69504165649414,1,60.5,0.0,0.0,15.3,15.75,0.3,91.85
2,2015-01-13 00:09:39,2015-01-13 01:03:11,1,28.66,-73.99616241455078,40.725425720214844,4,false,-73.80000305175781,40.964561462402344,2,83.5,0.5,0.5,0.0,2.44,0.3,87.24
2,2015-01-01 05:29:50,2015-01-01 05:54:13,1,17.46,-73.98296356201172,40.757415771484375,3,false,-74.17719268798828,40.6950569152832,1,64.0,0.5,0.0,1.0,16.0,0.3,81.8
1,2015-01-15 10:26:16,2015-01-15 10:26:42,2,0.0,-73.98765563964844,40.74364471435547,5,false,-73.98748779296875,40.74348449707031,1,60.0,0.0,0.0,15.0,0.0,0.3,75.3
2,2015-01-25 17:45:15,2015-01-25 17:45:21,1,0.0,-73.97565460205078,40.76131820678711,2,false,0.0,0.0,1,52.0,0.0,0.5,20.0,0.0,0.3,72.8
2,2015-01-15 14:00:43,2015-01-15 14:58:40,1,15.2,-73.80126190185547,40.667728424072266,2,false,-73.99335479736328,40.75600051879883,1,52.0,0.0,0.5,14.33,5.33,0.3,72.46
2,2015-01-02 19:09:23,2015-01-02 19:16:34,1,1.29,-73.99647521972656,40.71571350097656,5,false,-74.00939178466797,40.70545196533203,1,58.0,0.0,0.5,11.6,0.0,0.3,70.4
2,2015-01-13 00:09:33,2015-01-13 00:31:09,2,16.76,-73.79154968261719,40.6454963684082,2,false,-73.9756851196289,40.752197265625,1,52.0,0.0,0.5,11.47,5.33,0.3,69.6
1,2015-01-04 13:44:55,2015-01-04 14:16:21,1,18.1,-73.99667358398438,40.732364654541016,2,false,-73.78376770019531,40.64366149902344,1,52.0,0.0,0.5,13.12,0.0,0.0,65.92


## Aggregations
- Aggregations are always a useful function whether it be for summarising data for analysis or fact tables.
- Like Pandas, Spark also covers `count()`, `mean()`, `max()`, `agg()`, etc...

In [33]:
from pyspark.sql.functions import mean

small_sdf.groupBy("passenger_count") \
    .agg(mean("total_amount").alias("Average Trip Amount USD$"),
         mean("trip_distance").alias("Average Distance in Miles")) 

passenger_count,Average Trip Amount USD$,Average Distance in Miles
1,14.798684931506786,2.6749931600547203
6,16.66782608695653,3.2256521739130446
3,16.75059999999999,3.4102000000000006
5,16.352564102564102,3.0589743589743583
4,12.951666666666668,1.8166666666666667
2,16.9891095890411,3.021780821917808


In [34]:
# and yes, it does work on the full dataset (albeit it does take time...)
results = sdf_with_schema.groupBy("passenger_count") \
    .agg(mean("total_amount").alias('avg_trip_amount')) \
    .orderBy("passenger_count")

results

passenger_count,avg_trip_amount
0,13.033572474377738
1,14.667256653975395
2,15.37395457318419
3,14.876514480790858
4,14.968314051202968
5,14.864738578983076
6,14.564280093219365
7,16.295
8,58.050000000000004
9,21.076666666666668


## Writing to Disk
- Conventionally, the "go-to" dataset was a `csv`. For good reasons, we have explored alternatives such as data serialization methods with formats such as Python's `pickle` or Apache Arrow's `feather`.
- Spark introduces its' own type which is a **Parquet File**.
- You can write a specific format (if supported) using `sdf.write.format("parquet").save(path)`.

### Parquet:
- Parquet files are stored as a directory structure which contains data files, metadata, and some compressed files.|
- If the file already exists, it cannot be overwritten without removing the existing file.

In [49]:
# Check to see if the fpath already exists. If so, remove it.
from shutil import rmtree
from os import path

fpath = '../data/aggregated_results.parquet/'
if path.exists(fpath):
    rmtree(fpath)

In [50]:
results.write.format('parquet').save('../data/aggregated_results.parquet')

Reading in `parquet` files are similar to `csv`:

In [52]:
spark.read.parquet(fpath)

passenger_count,avg_trip_amount
0,13.033572474377738
1,14.667256653975395
2,15.37395457318419
3,14.876514480790858
4,14.968314051202968
5,14.864738578983076
6,14.564280093219365
7,16.295
8,58.050000000000004
9,21.076666666666668


## Union and Merging
- The `union()` method merges two Spark DataFrames and returns *a new* DataFrame with all rows from the two DataFrames *including duplicates*. 
- It works identical to SQL `UNION` and as a result, may include duplicate results.
- If you want no duplicates, you can do `union().distinct()` (distinct was mentioned previously).

The example below takes the union of two identical DataFrames consisting of 5 rows.

In [54]:
sdf1 = spark.createDataFrame(df.iloc[:5])
sdf2 = spark.createDataFrame(df.iloc[:5])

sdf1.union(sdf2)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,1/12/15 0:00,1/12/15 0:05,5,0.96,-73.97994232,40.76538086,1,N,-73.96630859,40.76308823,1,5.5,0.5,0.5,1.0,0.0,0.3,7.8
2,1/12/15 0:00,1/12/15 0:00,2,2.69,-73.97233582,40.76237869,1,N,-73.99362946,40.74599838,1,21.5,0.0,0.5,3.34,0.0,0.3,25.64
2,1/12/15 0:00,1/12/15 0:00,1,2.62,-73.96884918,40.76453018,1,N,-73.97454834,40.79164124,1,17.0,0.0,0.5,3.56,0.0,0.3,21.36
1,1/12/15 0:00,1/12/15 0:05,1,1.2,-73.99393463,40.74168396,1,N,-73.99766541,40.74746704,1,6.5,0.5,0.5,0.2,0.0,0.3,8.0
1,1/12/15 0:00,1/12/15 0:09,2,3.0,-73.98892212,40.72698975,1,N,-73.97559357,40.6968689,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3
2,1/12/15 0:00,1/12/15 0:05,5,0.96,-73.97994232,40.76538086,1,N,-73.96630859,40.76308823,1,5.5,0.5,0.5,1.0,0.0,0.3,7.8
2,1/12/15 0:00,1/12/15 0:00,2,2.69,-73.97233582,40.76237869,1,N,-73.99362946,40.74599838,1,21.5,0.0,0.5,3.34,0.0,0.3,25.64
2,1/12/15 0:00,1/12/15 0:00,1,2.62,-73.96884918,40.76453018,1,N,-73.97454834,40.79164124,1,17.0,0.0,0.5,3.56,0.0,0.3,21.36
1,1/12/15 0:00,1/12/15 0:05,1,1.2,-73.99393463,40.74168396,1,N,-73.99766541,40.74746704,1,6.5,0.5,0.5,0.2,0.0,0.3,8.0
1,1/12/15 0:00,1/12/15 0:09,2,3.0,-73.98892212,40.72698975,1,N,-73.97559357,40.6968689,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3


In [55]:
sdf1.union(sdf2).distinct()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,1/12/15 0:00,1/12/15 0:00,2,2.69,-73.97233582,40.76237869,1,N,-73.99362946,40.74599838,1,21.5,0.0,0.5,3.34,0.0,0.3,25.64
2,1/12/15 0:00,1/12/15 0:00,1,2.62,-73.96884918,40.76453018,1,N,-73.97454834,40.79164124,1,17.0,0.0,0.5,3.56,0.0,0.3,21.36
2,1/12/15 0:00,1/12/15 0:05,5,0.96,-73.97994232,40.76538086,1,N,-73.96630859,40.76308823,1,5.5,0.5,0.5,1.0,0.0,0.3,7.8
1,1/12/15 0:00,1/12/15 0:05,1,1.2,-73.99393463,40.74168396,1,N,-73.99766541,40.74746704,1,6.5,0.5,0.5,0.2,0.0,0.3,8.0
1,1/12/15 0:00,1/12/15 0:09,2,3.0,-73.98892212,40.72698975,1,N,-73.97559357,40.6968689,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3


# User Defined Functions (UDF) and Pandas UDFs
So far, all the functions and methods have been about simple aggregations or filtering rows. However, preprocessing and data cleansing usually requires more powerful tools such as `regex`.

Unlike Pandas's `apply()` method (and also `rdd.map()`), we need to do a "bit" more work to generate UDFs.

1. Create a function with a `@udf()` decorator.
2. Specify an output data type (i.e `StringType()`) as format `@udf("string")` or `@udf(StringType())`.
3. Apply onto column(s) of choice (remembering that Spark is immutable).

Alternatively, if we want to use Pandas framework:
1. Create a function with a `@pandas_udf()` decorator and format as required.
2. Apply onto column(s) of choice.

In the following example, we will create a tuple consisting of pickup lat/lon to 4 decimal places.

In [56]:
# using UDF
@F.udf(ArrayType(DoubleType(), True))
def create_coords(lat, lon):
    return round(lat, 4), round(lon, 4)

In [57]:
small_sdf.withColumn("pickup_coords", create_coords(col("pickup_latitude"), col("pickup_longitude"))) \
    .limit(10)

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_coords
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,false,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05,"[40.7501, -73.9939]"
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.3,-74.00164794921875,40.7242431640625,1,false,-73.99441528320312,40.75910949707031,1,14.5,0.5,0.5,2.0,0.0,0.3,17.8,"[40.7242, -74.0016]"
1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.8,-73.96334075927734,40.80278778076172,1,false,-73.95182037353516,40.82441329956055,2,9.5,0.5,0.5,0.0,0.0,0.3,10.8,"[40.8028, -73.9633]"
1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.5,-74.00908660888672,40.71381759643555,1,false,-74.00432586669922,40.71998596191406,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8,"[40.7138, -74.0091]"
1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.0,-73.97117614746094,40.762428283691406,1,false,-74.00418090820312,40.742652893066406,2,15.0,0.5,0.5,0.0,0.0,0.3,16.3,"[40.7624, -73.9712]"
1,2015-01-10 20:33:39,2015-01-10 20:53:52,1,9.0,-73.87437438964844,40.7740478515625,1,false,-73.98697662353516,40.75819396972656,1,27.0,0.5,0.5,6.7,5.33,0.3,40.33,"[40.774, -73.8744]"
1,2015-01-10 20:33:39,2015-01-10 20:58:31,1,2.2,-73.9832763671875,40.726009368896484,1,false,-73.99246978759766,40.7496337890625,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,"[40.726, -73.9833]"
1,2015-01-10 20:33:39,2015-01-10 20:42:20,3,0.8,-74.0026626586914,40.7341423034668,1,false,-73.99501037597656,40.72632598876953,1,7.0,0.5,0.5,1.66,0.0,0.3,9.96,"[40.7341, -74.0027]"
1,2015-01-10 20:33:39,2015-01-10 21:11:35,3,18.2,-73.78304290771484,40.64435577392578,2,false,-73.98759460449219,40.75935745239258,2,52.0,0.0,0.5,0.0,5.33,0.3,58.13,"[40.6444, -73.783]"
1,2015-01-10 20:33:40,2015-01-10 20:40:44,2,0.9,-73.98558807373047,40.767948150634766,1,false,-73.98591613769531,40.75936508178711,1,6.5,0.5,0.5,1.55,0.0,0.3,9.35,"[40.7679, -73.9856]"


And here's an example of mapping values from our data dictionary using a Pandas UDF:
- Type definition Syntax: https://www.python.org/dev/peps/pep-0484/#type-definition-syntax
- Function Decorators: https://johnpaton.net/posts/clean-spark-udfs/

The Pandas UDF is also quite new so there isn't much *help* other than the documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.pandas_udf.html?highlight=pandas%20udf

Syntax:
```python
@pandas_udf(THE DATATYPE OF THE OUTPUT)
def FUNCTION_NAME(ARGUMENTS: INPUT DATA FORMAT) -> OUTPUT DATA FORMAT:
    ...
    return ...

sdf.withColumn(COLUMN OUT, FUNCTION_NAME(col(COLUMN IN)))
```

In [58]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [59]:
vendors = {1: 'Creative Mobile Technologies, LLC', 2: 'VeriFone Inc.'}

@pandas_udf("string")
def vendorMap(vid_col: pd.Series) -> pd.Series:
    return vid_col.map(vendors)

In [60]:
small_sdf.withColumn("VendorName", vendorMap(col("VendorID"))) \
    .limit(10)

VendorID,pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,VendorName
2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896484375,40.7501106262207,1,false,-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05,VeriFone Inc.
1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.3,-74.00164794921875,40.7242431640625,1,false,-73.99441528320312,40.75910949707031,1,14.5,0.5,0.5,2.0,0.0,0.3,17.8,Creative Mobile T...
1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.8,-73.96334075927734,40.80278778076172,1,false,-73.95182037353516,40.82441329956055,2,9.5,0.5,0.5,0.0,0.0,0.3,10.8,Creative Mobile T...
1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.5,-74.00908660888672,40.71381759643555,1,false,-74.00432586669922,40.71998596191406,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8,Creative Mobile T...
1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.0,-73.97117614746094,40.762428283691406,1,false,-74.00418090820312,40.742652893066406,2,15.0,0.5,0.5,0.0,0.0,0.3,16.3,Creative Mobile T...
1,2015-01-10 20:33:39,2015-01-10 20:53:52,1,9.0,-73.87437438964844,40.7740478515625,1,false,-73.98697662353516,40.75819396972656,1,27.0,0.5,0.5,6.7,5.33,0.3,40.33,Creative Mobile T...
1,2015-01-10 20:33:39,2015-01-10 20:58:31,1,2.2,-73.9832763671875,40.726009368896484,1,false,-73.99246978759766,40.7496337890625,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,Creative Mobile T...
1,2015-01-10 20:33:39,2015-01-10 20:42:20,3,0.8,-74.0026626586914,40.7341423034668,1,false,-73.99501037597656,40.72632598876953,1,7.0,0.5,0.5,1.66,0.0,0.3,9.96,Creative Mobile T...
1,2015-01-10 20:33:39,2015-01-10 21:11:35,3,18.2,-73.78304290771484,40.64435577392578,2,false,-73.98759460449219,40.75935745239258,2,52.0,0.0,0.5,0.0,5.33,0.3,58.13,Creative Mobile T...
1,2015-01-10 20:33:40,2015-01-10 20:40:44,2,0.9,-73.98558807373047,40.767948150634766,1,false,-73.98591613769531,40.75936508178711,1,6.5,0.5,0.5,1.55,0.0,0.3,9.35,Creative Mobile T...


And that's the basics of PySpark! If you would like to further increase your scope, here are some pathways:
- Data Science: Continue with Spark's MLlib to perform machine learning.
- Data Engineering: Learn Spark SQL and Spark Connectors (i.e connecting to data sources such as S3 buckets)